In [1]:
import numpy as np
import pandas as pd
import joblib

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
import re
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

import joblib

In [19]:
df = pd.read_csv('training_set_7_final.txt', sep='\t', error_bad_lines=False, encoding="ISO-8859-1")

In [20]:
def text_to_cleantext_tokenizer(text, remove_nonletters = False, remove_stopwords=False, stemming=False, lemma=False):

    # 1. Remove HTML
    teks = BeautifulSoup(text, 'lxml').get_text()

    # 2. Remove non-ASCII
    letters_only = re.sub(r"[^\x00-\x7F]+", " ", teks)

    if remove_nonletters:
        letters_only = re.sub("[^a-zA-Z]", " ", letters_only)

    #letters_only = teks

    # 3. Convert to lower-case, split into words
    words = letters_only.lower()
    words = word_tokenize(words)

    # 4. Convert stopwords into Set (faster than List)
    # 5. Remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    # 6. Stemming
    if stemming:
        porter = PorterStemmer()
        stems = []
        for t in words:
            stems.append(porter.stem(t))
        words = stems

    # 7. Stemming
    if lemma:
        lemmatizer = WordNetLemmatizer()
        lemmas = []
        for t in words:
            lemmas.append(lemmatizer.lemmatize(t))
        words = lemmas
        
    # 8. Join words back into one string by space, and return the result
    return(" ".join(words))
    # 8. Return list of words
    # return(words)
    

In [21]:
df['essay'][0]

'Patience is when your waiting .I was patience when in line waiting for lunch .I didn\x92t c ut any one to eat .I was standing and waiting for my turn .Patience ,some people don\x92t have it .Lots of people just cut or yell at you because they don\x92t have  any patience. Sometimes people will push you out of their way .They only do that because they don\x92t have patience at all. Patience is what people need .People need patience because lots o f feelings get hurt .Everyone should have patience.'

In [22]:
for idx, row in df.iterrows():  
    line = df.at[idx, 'essay']
    line = line.encode('ascii','ignore')
    line = text_to_cleantext_tokenizer(line, remove_nonletters=True, remove_stopwords=True, lemma=True)
    df.at[idx,'essay'] = line

In [23]:
df['essay'][0]

'patience waiting patience line waiting lunch didnt c ut one eat standing waiting turn patience people dont lot people cut yell dont patience sometimes people push way dont patience patience people need people need patience lot f feeling get hurt everyone patience'

In [24]:
vectorizer = TfidfVectorizer(min_df=3, ngram_range=(1,2))
X = vectorizer.fit_transform(df['essay'])

In [25]:
len(vectorizer.get_feature_names_out())

8317

In [26]:
vectorizer.get_feature_names_out()

array(['ability', 'able', 'able get', ..., 'zoo', 'zoom', 'zoomed'],
      dtype=object)

In [27]:
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
cosine_similarity(X[625],X[880])

array([[0.0651608]])

In [29]:
cosine_similarity(X[59],X[1064])

array([[0.03732957]])

In [30]:
cosine_similarity(X[756],X[270])

array([[0.2975554]])

In [31]:
cosine_similarity(X[262],X[466])

array([[0.]])

In [32]:
joblib.dump(X.toarray(), 'essay_tfidf_no_stopword_asap7')

['essay_tfidf_no_stopword_asap7']

In [26]:
vect = TfidfVectorizer(ngram_range=(1,2))
X = vect.fit_transform(['Patience is important'])

In [27]:
X

<1x5 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [28]:
vect.vocabulary_

{'patience': 3, 'is': 1, 'important': 0, 'patience is': 4, 'is important': 2}